# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.txt** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **1. clean all outputs and 2. run all cells in your notebook and keep all running logs** so that we can check.

In [1]:
!pip install numpy==1.26.4 nltk==3.9.1 scikit-learn==1.6.1 gensim==4.3.3 setuptools

In [2]:
# Environment Setup and Validation
import sys
print(f"Python version: {sys.version}")

# Required package versions
required_packages = {
    'numpy': '1.26.4',
    'nltk': '3.9.1',
    'scikit-learn': '1.6.1',
    'gensim': '4.3.3'
}

def validate_environment():
    import pkg_resources
    installed = {pkg.key: pkg.version for pkg in pkg_resources.working_set}
    for package, min_version in required_packages.items():
        if package not in installed:
            print(f"❌ {package} not found!")
            return False
        if pkg_resources.parse_version(installed[package]) < pkg_resources.parse_version(min_version):
            print(f"❌ {package} version {installed[package]} is below minimum {min_version}")
            return False
        print(f"✓ {package} version {installed[package]}")
    return True

assert validate_environment(), "Please install required packages with correct versions"

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


<ipython-input-2-56c000555d47>:14: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


✓ numpy version 1.26.4
✓ nltk version 3.9.1
✓ scikit-learn version 1.6.1
✓ gensim version 4.3.3


# 1 $n$-gram Language Model

In [3]:
!mkdir -p data/lm
!wget -O data/lm/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/lm/train.txt
!wget -O data/lm/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/lm/dev.txt
!wget -O data/lm/test.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/lm/test.txt

--2025-02-28 16:36:22--  https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/lm/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8441444 (8.0M) [text/plain]
Saving to: ‘data/lm/train.txt’

data/lm/train.txt   100%[===================>]   8.05M  --.-KB/s    in 0.1s    

2025-02-28 16:36:22 (59.2 MB/s) - ‘data/lm/train.txt’ saved [8441444/8441444]

--2025-02-28 16:36:22--  https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/lm/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting r

## 1.1 Building vocabulary

### Instructions:
1. Implement a Vocabulary class that:
   - Handles OOV words (tokens occurring < 3 times)
   - Includes special tokens \<UNK>, \<START>, \<END>
   - Provides word2idx and idx2word mappings
2. Expected vocabulary size: 24,067 words

### Code

In [4]:
# 1.1 Building vocabulary
import nltk
from collections import Counter
from typing import List, Dict

class Vocabulary:
    def __init__(self):
        self.word2idx = {'<UNK>': 0, '<START>': 1, '<END>': 2}
        self.idx2word = ['<UNK>', '<START>', '<END>']
        self.word_counts = {}

    def build_vocab(self, filepath: str, min_freq: int = 3) -> None:
        """Build vocabulary from training file

        Args:
            filepath: Path to training file
            min_freq: Minimum frequency threshold for words
        """
        # TODO: Implement vocabulary building
        # Read and tokenize the input file
        content = ''
        with open(filepath, 'r') as f:
            for line in f:
                content += line.lower() + '\n'

        # Tokenize the text (whitespace-tokenized, as specified in the assignment)
        nltk.download('punkt_tab')
        words = nltk.word_tokenize(content)

        # Count word frequencies
        self.word_counts = Counter(words)
        unk_count = 0
        # Add words that meet frequency threshold to vocab
        for word, count in self.word_counts.items():
            if count >= min_freq and word not in self.word2idx:
                idx = len(self.word2idx)
                self.word2idx[word] = idx
                self.idx2word.append(word)
            else:
                unk_count += count
        self.word_counts['<UNK>'] = unk_count

    def __len__(self) -> int:
        return len(self.word2idx)  # Subtract 2 to exclude <START> and <END> from the count

    def get_id(self, word: str) -> int:
        """Get ID for word (returns UNK if not in vocab)"""
        # TODO: Implement
        return self.word2idx.get(word, self.word2idx['<UNK>'])

    def get_word(self, idx: int) -> str:
        """Get word for ID"""
        # TODO: Implement
        return self.idx2word[idx]

# Test vocabulary
vocab = Vocabulary()
vocab.build_vocab('data/lm/train.txt')
print(f"Vocabulary size: {len(vocab)}")
assert len(vocab) == 24067, "Vocabulary size should be 24,067"

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Vocabulary size: 24067


### Discussion

Discuss the number of parameters in n-gram models here

The number of parameters in n-gram models here is $V^n$, where $V$ is the size of the vocabulary and $n$ is the number of words in the n-gram. In this case, the number of parameters is $24067^n$.

The parameters are typically represented as conditional probabilities of the form:
$$ P(w_n | w_{n-1}, w_{n-2}, ..., w_{1}) $$

where $w_i$ is the $i$-th word in the n-gram. For each word $w_m$, there are $(n-1)$ words that precede it, and each of these words can be any of the $V$ words in the vocabulary. Therefore, there are $V^(n-1)$ possible combinations of the preceding words. Then, there are $V$ possible values for the word $w_m$. Therefore, there are $V\times V^{n-1}=V^n$ possible parameters in the n-gram model.

## 1.2 $n$-gram Language Modeling

### Instructions:
1. Implement an n-gram language model that:
   - Counts n-grams from training data
   - Computes probabilities
   - Calculates perplexity
2. Test with both unigram and bigram models
3. Report perplexity on train and dev sets

### Code

In [ ]:
import numpy as np
from collections import defaultdict
from typing import List, Tuple

class LanguageModel:
    def __init__(self, vocab: Vocabulary, n: int):
        """Initialize n-gram language model

        Args:
            vocab: Vocabulary object
            n: Order of n-gram model
        """
        self.vocab = vocab
        self.n = n
        self.ngram_counts = defaultdict(int)
        self.context_counts = defaultdict(int)

    def get_ngrams(self, tokens: List[str], n: int) -> List[Tuple[str]]:
        """Extract n-grams from token sequence"""
        # TODO: Implement n-gram extraction
        ngrams = []
        # Slide over the token list to generate n-grams
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            ngrams.append(ngram)
        return ngrams

    def train(self, filepath: str) -> None:
        """Train n-gram language model"""
        # TODO: Implement training
        with open(filepath, 'r') as f:
            text = f.read()

        sentences = text.splitlines()
        for sentence in sentences:
            words = sentence.split()
            words = [word.lower() for word in words]
            # Convert words to their corresponding ids using word2idx
            words = [self.vocab.get_id(word) for word in words]
            ngrams = self.get_ngrams(words, self.n)
            for ngram in ngrams:
                self.ngram_counts[ngram] += 1
                context = ngram[:-1]  # Context is the first n-1 words
                self.context_counts[context] += 1

    def get_prob(self, word: str, context: Tuple[int]) -> float:
        """Get probability of word given context"""
        word_id = self.vocab.get_id(word)
        context_count = self.context_counts[context]
        ngram_count = self.ngram_counts[context + (word_id,)]
        if self.n == 1:  # Unigram case
            return ngram_count / sum(self.ngram_counts.values())

        # For bigram or trigram, apply smoothing and probability calculation
        prob = ngram_count / context_count
        if prob == 0:
            return 1 / len(self.vocab)+context_count  # Smoothing (add-one)
        return prob

    def perplexity(self, filepath: str) -> float:
        """Calculate perplexity on given text"""
        total_log_prob = 0
        total_words = 0
        with open(filepath, 'r') as f:
            text = f.read()

        sentences = text.splitlines()
        for sentence in sentences:
            total_words += len(sentence.split())
            words = sentence.split()
            words = [word.lower() for word in words]
            # Convert words to their corresponding ids using word2idx
            words = [self.vocab.get_id(word) for word in words]
            ngrams = self.get_ngrams(words, self.n)
            for ngram in ngrams:
                context = ngram[:-1]
                word_id = ngram[-1]
                prob = self.get_prob(self.vocab.get_word(word_id), context)
                #if prob == 0:
                #    print(f"Zero probability for {self.vocab.get_word(word_id)} given context {context}")
                total_log_prob += np.log(prob)

        l = total_log_prob / total_words
        perplexity = 2 ** -l
        return perplexity

# Test language model
lm_2 = LanguageModel(vocab, n=2)
lm_2.train('data/lm/train.txt')
dev_perplexity = lm_2.perplexity('data/lm/dev.txt')
print(f"Perplexity on dev (bigram): {dev_perplexity:.2f}")
assert dev_perplexity > 0, "Perplexity should be positive"
train_perplexity_2 = lm_2.perplexity('data/lm/train.txt')
print(f"Perplexity on train (bigram): {train_perplexity_2:.2f}")

lm_1 = LanguageModel(vocab, n=1)
lm_1.train('data/lm/train.txt')
dev_perplexity_1 = lm_1.perplexity('data/lm/dev.txt')
print(f"Perplexity on dev (unigram): {dev_perplexity_1:.2f}")
train_perplexity_1 = lm_1.perplexity('data/lm/train.txt')
print(f"Perplexity on train (unigram): {train_perplexity_1:.2f}")

Perplexity on dev (bigram): 4.85
Perplexity on train (bigram): 20.26
Perplexity on dev (unigram): 109.23


### Discussion

Compare unigram and bigram model performance here

## 1.3 Smoothing

### 1.3.1 Add-one (Laplace) smoothing

### Code

In [ ]:
class AddOneLanguageModel(LanguageModel):
    def get_prob(self, word: str, context: Tuple[str]) -> float:
        """Get smoothed probability using add-one smoothing"""
        # TODO: Implement add-one smoothing
        pass

# Test add-one smoothing
add_one_lm = AddOneLanguageModel(vocab, n=2)
add_one_lm.train('data/lm/train.txt')
print(f"Add-one perplexity on dev: {add_one_lm.perplexity('data/lm/dev.txt'):.2f}")

#### Discussion

Analyze how add-one smoothing affects the model performance

#### 1.3.2 Add-k smoothing

### Instructions:
1. Implement add-k smoothing with configurable k
2. Try k values: 0.1, 0.5, 1.0

##### Code

In [ ]:
class AddKLanguageModel(LanguageModel):
    def __init__(self, vocab: Vocabulary, n: int, k: float):
        super().__init__(vocab, n)
        self.k = k

    def get_prob(self, word: str, context: Tuple[str]) -> float:
        """Get smoothed probability using add-k smoothing"""
        # TODO: Implement add-k smoothing
        pass

# Test add-k smoothing with different k values
k_values = [0.1, 0.5, 1.0]
for k in k_values:
    add_k_lm = AddKLanguageModel(vocab, n=2, k=k)
    add_k_lm.train('data/lm/train.txt')
    print(f"Add-{k} perplexity on dev: {add_k_lm.perplexity('data/lm/dev.txt'):.2f}")

##### Discussion

Compare performance with different k values

### 1.3.3 Linear Interpolation

### Instructions:
1. Implement linear interpolation of unigram, bigram, and trigram models
2. Initial lambdas: [0.1, 0.3, 0.6]

#### Code

In [ ]:
class InterpolatedLanguageModel:
    def __init__(self, vocab: Vocabulary, lambdas: List[float]):
        """
        Args:
            vocab: Vocabulary object
            lambdas: List of interpolation weights (should sum to 1)
        """
        assert abs(sum(lambdas) - 1.0) < 1e-6, "Lambdas must sum to 1"
        self.vocab = vocab
        self.lambdas = lambdas
        self.models = [LanguageModel(vocab, n=i+1) for i in range(len(lambdas))]

    def train(self, filepath: str) -> None:
        """Train all n-gram models"""
        for model in self.models:
            model.train(filepath)

    def get_prob(self, word: str, context: Tuple[str]) -> float:
        """Get interpolated probability"""
        # TODO: Implement interpolated probability
        pass

    def perplexity(self, filepath: str) -> float:
        """Calculate perplexity using interpolated probabilities"""
        # TODO: Implement perplexity calculation
        pass

# Test interpolated model
lambdas = [0.1, 0.3, 0.6]  # Example weights for unigram, bigram, trigram
interpolated_lm = InterpolatedLanguageModel(vocab, lambdas)
interpolated_lm.train('data/lm/train.txt')
print(f"Interpolated perplexity on dev: {interpolated_lm.perplexity('data/lm/dev.txt'):.2f}")

#### Discussion

Analyze the effectiveness of interpolation

##### 1.3.4 Optimization

#### Discussion

# 2 Word Vectors

In [ ]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each lengh 50
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin

wv_from_bin = load_embedding_model()

## 2.1 Find most similar word
### Instructions:
1. Implement function to find similar words using cosine similarity
2. Expected similarity scores should be between 0 and 1
3. Return top 3 similar words for each query

In [ ]:
def find_most_similar(word: str, wv_from_bin) -> List[Tuple[str, float]]:
    """Find most similar words using cosine similarity

    Args:
        word: Query word
        wv_from_bin: Loaded word vectors

    Returns:
        List of (word, similarity) tuples
    """
    # TODO: Implement similarity search
    pass

test_words = [
    'ubiquitous',
    'serendipity',
    'melancholy',
    'paradox',
    'ethereal',
    'cacophony'
]
for word in test_words:
    similar_words = find_most_similar(word, wv_from_bin)
    print(f"\nMost similar to '{word}':")
    for similar_word, similarity in similar_words:
        print(f"{similar_word}: {similarity:.3f}")
        assert 0 <= similarity <= 1, f"Invalid similarity score: {similarity}"

## 2.2 Finding Analogies
Use vector addition and subtraction to compute target vectors for the analogies below. After computing each target vector, find the top three candidates by cosine similarity. Report the candidates and their similarities to the target vector.

- dog : puppy :: cat : ?:
- speak : speaker :: sing : ?:
- France : French :: England : ?:
- France : wine :: England : ?

In [ ]:
def find_analogy(a: str, b: str, c: str, wv_from_bin) -> List[Tuple[str, float]]:
    """Find word analogies using vector arithmetic

    Args:
        a, b, c: Words for analogy a:b :: c:?
        wv_from_bin: Loaded word vectors

    Returns:
        List of (word, similarity) tuples
    """
    # TODO: Implement analogy finding
    pass

# Test analogies
analogies = [
    ('king', 'queen', 'man'),
    ('paris', 'france', 'rome'),
    ('france', 'french', 'germany'),
    ('london', 'england', 'paris'),
    ('cat', 'kitten', 'dog'),
    ('rich', 'poor', 'strong')
]
for a, b, c in analogies:
    print(f"\nAnalogy: {a}:{b} :: {c}:?")
    results = find_analogy(a, b, c, wv_from_bin)
    for word, similarity in results:
        print(f"{word}: {similarity:.3f}")
        assert 0 <= similarity <= 1, f"Invalid similarity score: {similarity}"

# 3 Sentiment analysis

In [ ]:
!mkdir -p data/classification
!wget -O data/classification/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/classification/train.txt
!wget -O data/classification/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/classification/dev.txt
!wget -O data/classification/test-blind.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/classification/test-blind.txt

## 3.1 Logistic Regression

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np

def load_data(filepath: str) -> Tuple[List[str], List[int]]:
    """Load text and labels from file"""
    texts, labels = [], []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            label, text = line.strip().split('\t')
            texts.append(text)
            labels.append(int(label))
    return texts, labels

class SentimentClassifier:
    def __init__(self, feature_type: str = 'unigram'):
        """
        Args:
            feature_type: One of 'unigram', 'bigram', or 'glove'
        """
        self.feature_type = feature_type
        self.vectorizer = None
        self.classifier = LogisticRegression(random_state=42)

    def extract_features(self, texts: List[str]) -> np.ndarray:
        """Extract features based on feature_type"""
        if self.feature_type == 'unigram':
            # TODO: Implement unigram feature extraction
            pass
        elif self.feature_type == 'bigram':
            # TODO: Implement bigram feature extraction
            pass
        elif self.feature_type == 'glove':
            # TODO: Implement glove feature extraction
            # Average word vectors for each text
            pass
        else:
            raise ValueError(f"Invalid feature type: {self.feature_type}")

    def train(self, train_texts: List[str], train_labels: List[int]) -> None:
        """Train sentiment classifier"""
        # TODO: Implement training
        pass

    def predict(self, texts: List[str]) -> np.ndarray:
        """Predict sentiment labels"""
        # TODO: Implement prediction
        pass

    def evaluate(self, texts: List[str], labels: List[int]) -> Dict:
        """Evaluate classifier performance"""
        predictions = self.predict(texts)
        return classification_report(labels, predictions, output_dict=True)

# Train and evaluate models with different features
train_texts, train_labels = load_data('data/classification/train.txt')
dev_texts, dev_labels = load_data('data/classification/dev.txt')

results = {}
for feature_type in ['unigram', 'bigram', 'glove']:
    classifier = SentimentClassifier(feature_type)
    classifier.train(train_texts, train_labels)
    results[feature_type] = classifier.evaluate(dev_texts, dev_labels)

# Print results
for feature_type, metrics in results.items():
    print(f"\n{feature_type.upper()} Results:")
    print(f"Precision: {metrics['weighted avg']['precision']:.3f}")
    print(f"Recall: {metrics['weighted avg']['recall']:.3f}")
    print(f"F1-score: {metrics['weighted avg']['f1-score']:.3f}")

Compare the performance of three types of features on dev set. Report the weighted average precision, recall and F1-score for each feature set.

| Feature | precision | recall | F1-score |
| ----------- | --------- | ------ | -------- |
| unigram     |           |        |          |
| bigram      |           |        |          |
| GloVe       |           |        |          |

## 3.2 Better Feature